In [1]:
import numpy as np
from scipy.signal import butter, filtfilt
from scipy.io import loadmat
from matplotlib import pyplot as plt
from pprint import pprint

import os

rootdir = ".." if os.getcwd().endswith("tests") else "."

from oct2py import octave

octave.addpath(os.path.join(rootdir, "LeTalker1.22"))
octave.eval("warning ('off', 'Octave:data-file-in-path')")

In [2]:
vt_data = loadmat(
    os.path.join(rootdir, "src", "simvoice", "data", "bs_origvowels.mat")
) | loadmat(os.path.join(rootdir, "src", "simvoice", "data", "LeTalkerTrachea.mat"))
pprint(vt_data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'aa', 'ii', 'uu', 'ae', 'ih', 'eh', 'ah', 'aw', 'uh', 'oo', 'uumod', 'areas', 'vow_order', 'note', 'trach'])


In [3]:
areas = vt_data["aa"][0]
atten = 0.005

# areas = areas[:6]

K = len(areas)
M = K // 2  # number of states for forward and backward pressures

if M * 2 != K:
    raise ValueError("tube must have an even number of crosssectional areas.")

f_even = np.zeros(M)  # [F2, F4, ..., FK]
b_odd = np.zeros(M)  # [B1, B3, ..., BK-1]

# f_odd # [F3, F5, ..., FK-1]
# b_even # [B2, B4, ..., BK-2]

a = areas
alpha = 1 - atten / (areas) ** 0.5
r = (a[:-1] - a[1:]) / (a[:-1] + a[1:])
rp1 = 1 + r
rm1 = 1 - r

M -= 1

Rodd = np.zeros((2 * M, 2 * M))
Rodd[1:M, : M - 1] = np.diag(rp1[2:-2:2] * alpha[2:-2:2])
Rodd[:M, M:] = np.diag(-r[:-1:2] * alpha[1:-2:2])
Rodd[M:, :M] = np.diag(r[2::2] * alpha[2::2])
Rodd[M:-1, M + 1 :] = np.diag(rm1[2:-2:2] * alpha[3:-1:2])
Reven = np.zeros((2 * M, 2 * M))
Reven[:M, :M] = np.diag(rp1[1:-1:2] * alpha[1:-1:2])
Reven[:M, M:] = np.diag(-r[1:-1:2] * alpha[2::2])
Reven[M:, :M] = np.diag(r[1:-1:2] * alpha[1:-1:2])
Reven[M:, M:] = np.diag(rm1[1:-1:2] * alpha[2::2])
A = Rodd @ Reven

print((A == 0).mean())

B = np.zeros((K, 2))
B[0, 0] = rp1[0] * alpha[0]
B[M, 0] = r[0] * alpha[0]
B[M - 1, 1] = -r[-1] * alpha[-1]
B[-1, 1] = rm1[-1] * alpha[-1]

C = np.zeros((2, 2 * M))
C[0, M - 1] = rp1[-1] * alpha[-2]
C[1, M] = rm1[0] * alpha[1]

D = np.zeros((2, 2))
D[0, 1] = -r[-1] * alpha[-1]
D[1, 0] = r[0] * alpha[0]

0.9070294784580499
